In [1]:
import pandas as pd

import pymorphy3

In [2]:
classification = pd.read_excel("../data/train_Ametist/classification.xlsx")
train_data = pd.read_excel("../data/train_Ametist/train.xlsx")

In [3]:
def extract_group(text, taget="Группа"):
    text = str(text)
    if taget in text:
        return text.split(":")[-1].replace(f"{taget} ", "").strip()
    return None

In [4]:
classification["group"] = classification["Классификатор строительных ресурсов"].apply(
    lambda x: extract_group(x, "Группа")
)
classification["group"] = classification["group"].fillna(method="ffill")

In [5]:
classification["book"] = classification["Классификатор строительных ресурсов"].apply(
    lambda x: extract_group(x, "Книга")
)
classification["book"] = classification["book"].fillna(method="ffill")

In [6]:
classification["part"] = classification["Классификатор строительных ресурсов"].apply(
    lambda x: extract_group(x, "Раздел")
)
classification["part"] = classification["part"].fillna(method="ffill")

In [7]:
clear_mask = (
    classification.dropna(axis=0, how="any")
    .iloc[:, 0]
    .apply(lambda x: all(c in "0123456789.- " for c in x))
)

cleared_classification = classification.dropna(axis=0, how="any")[clear_mask].rename(
    columns={
        "Классификатор строительных ресурсов": "code",
        "Unnamed: 1": "name",
        "Unnamed: 2": "measure",
    }
)

In [8]:
cleared_classification = cleared_classification[
    cleared_classification["code"].apply(lambda x: len(x.split("."))) > 4
].reset_index(drop=True)
cleared_classification.head()

,code,name,measure,group,book,part
0,23.65.12.190.01.1.01.01-0002-000,Детали фасонные коньковые к листам хризотилцем...,100 компл,Детали фасонные к листам хризотилцементным,Материалы для строительных и дорожных работ,"Материалы, изделия и конструкции хризотилсодер..."
1,23.65.12.190.01.1.01.02-0011-000,"Доска электротехническая дугостойкая (АЦЭИД), ...",т,Доски электротехнические,Материалы для строительных и дорожных работ,"Материалы, изделия и конструкции хризотилсодер..."
2,23.65.12.111.01.1.01.04-1018-000,"Листы хризотилцементные волнистые, профиль 40/...",м2,Листы хризотилцементные волнистые,Материалы для строительных и дорожных работ,"Материалы, изделия и конструкции хризотилсодер..."
3,23.65.12.111.01.1.01.04-1022-000,"Листы хризотилцементные волнистые, профиль 40/...",м2,Листы хризотилцементные волнистые,Материалы для строительных и дорожных работ,"Материалы, изделия и конструкции хризотилсодер..."
4,23.65.12.111.01.1.01.04-1024-000,"Листы хризотилцементные волнистые, профиль 40/...",м2,Листы хризотилцементные волнистые,Материалы для строительных и дорожных работ,"Материалы, изделия и конструкции хризотилсодер..."


In [9]:
clsf = cleared_classification

In [11]:
morph = pymorphy3.MorphAnalyzer()
lemm_texts_list = []

for text in clsf["name"]:
    text_lem = [morph.parse(word)[0].normal_form for word in text.split(" ")]
    if len(text_lem) <= 2:
        lemm_texts_list.append("")
        continue
    lemm_texts_list.append(" ".join(text_lem))
clsf["text_lemm"] = lemm_texts_list
clsf = clsf[clsf["text_lemm"] != ""]
clsf.head()

,code,name,measure,group,book,part,text_lemm
0,23.65.12.190.01.1.01.01-0002-000,Детали фасонные коньковые к листам хризотилцем...,100 компл,Детали фасонные к листам хризотилцементным,Материалы для строительных и дорожных работ,"Материалы, изделия и конструкции хризотилсодер...",деталь фасонный коньковый к лист хризотилцемен...
1,23.65.12.190.01.1.01.02-0011-000,"Доска электротехническая дугостойкая (АЦЭИД), ...",т,Доски электротехнические,Материалы для строительных и дорожных работ,"Материалы, изделия и конструкции хризотилсодер...","доска электротехнический дугостойкий (ацэид), ..."
2,23.65.12.111.01.1.01.04-1018-000,"Листы хризотилцементные волнистые, профиль 40/...",м2,Листы хризотилцементные волнистые,Материалы для строительных и дорожных работ,"Материалы, изделия и конструкции хризотилсодер...","лист хризотилцементный волнистые, профиль 40/1..."
3,23.65.12.111.01.1.01.04-1022-000,"Листы хризотилцементные волнистые, профиль 40/...",м2,Листы хризотилцементные волнистые,Материалы для строительных и дорожных работ,"Материалы, изделия и конструкции хризотилсодер...","лист хризотилцементный волнистые, профиль 40/1..."
4,23.65.12.111.01.1.01.04-1024-000,"Листы хризотилцементные волнистые, профиль 40/...",м2,Листы хризотилцементные волнистые,Материалы для строительных и дорожных работ,"Материалы, изделия и конструкции хризотилсодер...","лист хризотилцементный волнистые, профиль 40/1..."


In [12]:
del lemm_texts_list

In [83]:
print(clsf["book"].nunique())
print(clsf["part"].nunique())
print(clsf["group"].nunique())

39
463
2076


In [137]:
from sklearn.model_selection import train_test_split
import re


def cut_code(code):
    split_code = re.split(r"\.|-", code)
    return int("".join(split_code[:-9]))


X = clsf["text_lemm"]
y = clsf["code"].apply(lambda x: cut_code(x))
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2)

In [138]:
from catboost import CatBoostClassifier, Pool


def fit_model(train_pool, test_pool, **kwargs):
    model = CatBoostClassifier(
        task_type="GPU", eval_metric="TotalF1", od_type="Iter", od_wait=10, **kwargs
    )

    return model.fit(
        train_pool, eval_set=test_pool, verbose=1, plot=False, use_best_model=True
    )

In [139]:
train_pool = Pool(
    data=X_train,
    label=y_train,
    text_features=[X_train.name],
    feature_names=[X_train.name],
)
valid_pool = Pool(
    data=X_test.reset_index(drop=True),
    label=y_test,
    text_features=[X_train.name],
    feature_names=[X_train.name],
)

In [140]:
pd.concat([X_train, y_train], axis=1).head(100)

,text_lemm,code
87892,"кабель волоконно-оптический, повить стеклоните...",27
74,"прокладка паронитовый для опорный кронштейна, ...",23
100711,опора унифицировать стальной анкерно-угловой с...,25
47394,плита опорный для трубчатый винтовой штанга 30...,25
6056,смесь сухой теплоизоляционный на основа полист...,23
...,...,...
3883,"борфреза тип f, диаметр режущий часть 16 мм, д...",25
94181,переход бесшовный эксцентрический из нержавеющ...,24
72285,"клапан запорный с.кзсг 15-00-00-эн, исполнение...",28
15903,"проволока латунный л68, круглая, твердая, норм...",24


In [141]:
model = fit_model(
    train_pool,
    valid_pool,
    learning_rate=0.1,
    iterations=300,
    # dictionaries=[{"dictionary_id": "Word", "max_dictionary_size": "10000"}],
    # feature_calcers=["BoW:top_tokens_count=10000"],
)

0:	learn: 0.8732124	test: 0.8844353	best: 0.8844353 (0)	total: 326ms	remaining: 1m 37s
1:	learn: 0.8750885	test: 0.8854817	best: 0.8854817 (1)	total: 414ms	remaining: 1m 1s
2:	learn: 0.8750504	test: 0.8861389	best: 0.8861389 (2)	total: 509ms	remaining: 50.4s
3:	learn: 0.8921883	test: 0.9046559	best: 0.9046559 (3)	total: 596ms	remaining: 44.1s
4:	learn: 0.8921422	test: 0.9044540	best: 0.9046559 (3)	total: 683ms	remaining: 40.3s
5:	learn: 0.8934968	test: 0.9051131	best: 0.9051131 (5)	total: 770ms	remaining: 37.7s
6:	learn: 0.8926400	test: 0.9041135	best: 0.9051131 (5)	total: 859ms	remaining: 36s
7:	learn: 0.8929060	test: 0.9041349	best: 0.9051131 (5)	total: 947ms	remaining: 34.6s
8:	learn: 0.8927472	test: 0.9040598	best: 0.9051131 (5)	total: 1.03s	remaining: 33.4s
9:	learn: 0.9047360	test: 0.9171304	best: 0.9171304 (9)	total: 1.12s	remaining: 32.5s
10:	learn: 0.9048988	test: 0.9172613	best: 0.9172613 (10)	total: 1.21s	remaining: 31.8s
11:	learn: 0.9053905	test: 0.9171293	best: 0.9172613 

In [102]:
val_data = pd.read_excel("../data/train_Ametist/train.xlsx")
val_data.head()

,record_name,record_name_2,ref_code,ref_name,ref_unit
0,DIN -рейка оцинкованная ТН35-7.5 100 см (Chint),"DIN-рейка 35х7,5 мм длиной 1000 мм",27.33.13.130.20.2.08.01-0001-000,"DIN-рейки металлические, оцинкованные, размеры...",100 шт
1,Анкерный элемент ТехноНиколь 8*45мм,Анкерный элемент TN 8x4.5,25.94.11.190.01.7.15.01-1169-000,"Анкер грунтовый забивной самораскрывающийся, о...",шт
2,Анкер забивной М10 DRM 12x40 сталь,Анкер втулочный M10,25.94.11.190.01.7.15.01-0037-000,"Анкер забивной латунный, диаметр внутренней ре...",шт
3,Анкер забивной М10/12x40,Анкер втулочный M10,25.94.11.190.01.7.15.01-0037-000,"Анкер забивной латунный, диаметр внутренней ре...",шт
4,Анкер забивной М8 LAZ латунь,Анкер втулочный M8,25.94.11.190.01.7.15.01-0036-000,"Анкер забивной латунный, диаметр внутренней ре...",шт


In [89]:
lemm_texts_list = []

for text in val_data["ref_name"]:
    text_lem = [morph.parse(word)[0].normal_form for word in text.split(" ")]
    if len(text_lem) <= 2:
        lemm_texts_list.append("")
        continue
    lemm_texts_list.append(" ".join(text_lem))
val_data["text_lemm"] = lemm_texts_list
val_data = val_data[val_data["text_lemm"] != ""]
val_data.head()

,record_name,record_name_2,ref_code,ref_name,ref_unit,text_lemm
0,DIN -рейка оцинкованная ТН35-7.5 100 см (Chint),"DIN-рейка 35х7,5 мм длиной 1000 мм",27.33.13.130.20.2.08.01-0001-000,"DIN-рейки металлические, оцинкованные, размеры...",100 шт,"din-рейка металлические, оцинкованные, размер ..."
1,Анкерный элемент ТехноНиколь 8*45мм,Анкерный элемент TN 8x4.5,25.94.11.190.01.7.15.01-1169-000,"Анкер грунтовый забивной самораскрывающийся, о...",шт,"анкер грунтовый забивной самораскрывающийся, о..."
2,Анкер забивной М10 DRM 12x40 сталь,Анкер втулочный M10,25.94.11.190.01.7.15.01-0037-000,"Анкер забивной латунный, диаметр внутренней ре...",шт,"анкер забивной латунный, диаметр внутренний ре..."
3,Анкер забивной М10/12x40,Анкер втулочный M10,25.94.11.190.01.7.15.01-0037-000,"Анкер забивной латунный, диаметр внутренней ре...",шт,"анкер забивной латунный, диаметр внутренний ре..."
4,Анкер забивной М8 LAZ латунь,Анкер втулочный M8,25.94.11.190.01.7.15.01-0036-000,"Анкер забивной латунный, диаметр внутренней ре...",шт,"анкер забивной латунный, диаметр внутренний ре..."


In [144]:
model.feature_names_

['text_lemm']

In [143]:
# test_pool = Pool(
#     data=clsf["text_lemm"].reset_index(drop=True),
#     text_features=["text_lemm"],
#     feature_names=["text_lemm"],
# )
cnt = 0
for i, x in enumerate(model.predict(train_pool)):
    if int(x[0]) == int(clsf["code"].iloc[i].split(".")[0]):
        cnt += 1
print(cnt / len(X_train))

0.16405467713014926


In [114]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [1]:
logreg = Pipeline(
    [
        ("vect", CountVectorizer(analyzer="char", ngram_range=(2, 10))),
        ("tfidf", TfidfTransformer()),
    ]
)

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

print(classification_report(y_test, y_pred))
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted')}")

NameError: name 'Pipeline' is not defined